In [4]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import pandas as pd
import numpy as np
import matplotlib

In [7]:
# url_imagen = '/home/diego/Proyecto Vivero/data/01_VIVERO/01_imagenes/imagen_mesones_rgb.tif'
# shp = '/home/diego/Proyecto Vivero/data/01_VIVERO/02_puntos_plantas/plantas_mesones_2.shp'
# url_salida = "/home/diego/Proyecto Vivero/train2"
url_imagen = 'D:/Memoria/Codigo/AraucoNursery/PROYECTOS/VIVERO/imagenes/imagen_mesones_rgb.tif'
shp = 'D:/Memoria/Codigo/AraucoNursery/PROYECTOS/VIVERO/puntos_plantas/plantas_mesones_2.shp'
url_salida = "D:/Memoria/Codigo/AraucoNursery/train"

In [8]:
puntos = gpd.read_file(shp)
raster = rasterio.open(url_imagen)

In [9]:
offsetX = 0
offsetY = 0
width = 100
height = 100
step = 10
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [14]:
contador = 0
while (1):
        # finalizar si se a recorrido toda la imagen
    if(offsetY >= raster.height and offsetX >= raster.width):
        break
     #cuando llegue al borde en el eje x, debo bajar(sumar en eje y) y debo volver a empezar setear offsetX en 0 y widthX en 100
    elif(offsetX >= raster.width):
        offsetY = offsetY + step
        offsetX = 0
   
    #entregamos los pixeles para obtener las coordenadas
    t = raster.xy(offsetX, offsetY)
    l = raster.xy(offsetX + width , offsetY)
    r = raster.xy(offsetX, offsetY + height)
    b = raster.xy(offsetX + width , offsetY + height)  
    bbox = box(l[0], b[1], r[0], t[1])
        
    try:
        #transformacion de la imagen
        out_img, out_transform = mask(raster, shapes=[bbox], crop=True)
        valores = np.unique(out_img)
        # si solo se tiene valores NODATA en la imagen, la descartamos
        if np.array_equal(valores,[65535]):
            offsetX = offsetX + step
            contador = contador + 1
            continue
            
        matplotlib.image.imsave('{}/salida{}.png'.format(url_salida,contador), out_img.T)

    #interseccion ptos
        cantidad=puntos["geometry"].intersects(bbox).value_counts()
        print(cantidad)
        if (True in cantidad.index):
            numero = cantidad[True]
        else:
            numero = 0
        cuentas.loc[contador] = ["salida"+str(contador)+".png",numero]
        if(contador%1000 == 0):
            print("salida"+str(contador)+".png")

        
    except ValueError:
        print(url_salida+"/salida"+str(contador)+".pngtif")
#         print(coords)
    contador = contador + 1
    offsetX = offsetX + step 
   

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


/home/diego/Proyecto Vivero/train2/salida102.tif
/home/diego/Proyecto Vivero/train2/salida103.tif
/home/diego/Proyecto Vivero/train2/salida104.tif
/home/diego/Proyecto Vivero/train2/salida105.tif
/home/diego/Proyecto Vivero/train2/salida106.tif
/home/diego/Proyecto Vivero/train2/salida107.tif
/home/diego/Proyecto Vivero/train2/salida108.tif
/home/diego/Proyecto Vivero/train2/salida109.tif
/home/diego/Proyecto Vivero/train2/salida110.tif
/home/diego/Proyecto Vivero/train2/salida111.tif
/home/diego/Proyecto Vivero/train2/salida112.tif
/home/diego/Proyecto Vivero/train2/salida113.tif
/home/diego/Proyecto Vivero/train2/salida114.tif
/home/diego/Proyecto Vivero/train2/salida115.tif
/home/diego/Proyecto Vivero/train2/salida116.tif
/home/diego/Proyecto Vivero/train2/salida117.tif
/home/diego/Proyecto Vivero/train2/salida118.tif
/home/diego/Proyecto Vivero/train2/salida119.tif
/home/diego/Proyecto Vivero/train2/salida120.tif
/home/diego/Proyecto Vivero/train2/salida121.tif
/home/diego/Proyecto

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


False    40122
dtype: int64
False    40122
dtype: int64


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


False    40122
dtype: int64
False    40122
dtype: int64


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


False    40122
dtype: int64
False    40122
dtype: int64


KeyboardInterrupt: 

In [51]:
cuentas.to_csv(url_salida+"/cuentas.csv")